# Section Four - Slicing through noisy data to find high level groupings

To present our data in an understandable way and a manageable volume, we need to group and aggregate our data using different cross sections.

In [2]:
import pandas as pd
import numpy as np

# Splitting and clustering seemingly random data points

In [19]:
df = pd.DataFrame(
    {'A': np.arange(8), 'B': np.random.randn(8)},
    index=pd.MultiIndex.from_arrays(
          [['bar', 'bar', 'bar', 'bar', 'foo', 'foo', 'qux', 'qux'],
          ['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux']],
        names=['X', 'Y'])
)
df                

A         B
X   Y               
bar bar  0  1.153056
    bar  1 -0.394327
    baz  2  0.386976
    baz  3  0.128757
foo foo  4 -0.529231
    foo  5 -1.562874
qux qux  6 -0.380523
    qux  7 -0.392618

In [24]:
grouped = df.groupby('X')
for name, group in grouped:
    print(f"Group: {name}")
    display(group)

Group: bar


A         B
X   Y               
bar bar  0  1.153056
    bar  1 -0.394327
    baz  2  0.386976
    baz  3  0.128757

Group: foo


A         B
X   Y               
foo foo  4 -0.529231
    foo  5 -1.562874

Group: qux


A         B
X   Y               
qux qux  6 -0.380523
    qux  7 -0.392618

In [25]:
grouped = df.groupby(['X', 'Y'])
for name, group in grouped:
    print(f"Group: {name}")
    display(group)

Group: ('bar', 'bar')


A         B
X   Y               
bar bar  0  1.153056
    bar  1 -0.394327

Group: ('bar', 'baz')


A         B
X   Y               
bar baz  2  0.386976
    baz  3  0.128757

Group: ('foo', 'foo')


A         B
X   Y               
foo foo  4 -0.529231
    foo  5 -1.562874

Group: ('qux', 'qux')


A         B
X   Y               
qux qux  6 -0.380523
    qux  7 -0.392618

In [26]:
grouped.get_group(('bar', 'bar'))

A         B
X   Y               
bar bar  0  1.153056
    bar  1 -0.394327

In [27]:
grouped.aggregate(np.sum)

A         B
X   Y                
bar bar   1  0.758730
    baz   5  0.515733
foo foo   9 -2.092105
qux qux  13 -0.773142

In [28]:
# equivalent to this
df.groupby(['X', 'Y']).sum()

A         B
X   Y                
bar bar   1  0.758730
    baz   5  0.515733
foo foo   9 -2.092105
qux qux  13 -0.773142

In [29]:
# get summary stats
df.groupby(['X']).aggregate([np.mean, np.std, np.min, np.max])

A                             B                              
    mean       std amin amax      mean       std      amin      amax
X                                                                   
bar  1.5  1.290994    0    3  0.318616  0.644282 -0.394327  1.153056
foo  4.5  0.707107    4    5 -1.046052  0.730896 -1.562874 -0.529231
qux  6.5  0.707107    6    7 -0.386571  0.008552 -0.392618 -0.380523

# Resolving incorrect data collection with lambdas and functions

In [39]:
# standard numpy/pandas functions are not enough
grouped = df.groupby("X")
grouped.aggregate([np.sum, lambda x: np.std(x, ddof=1)])

A                   B          
    sum  <lambda>       sum  <lambda>
X                                    
bar   6  1.290994  1.274463  0.644282
foo   9  0.707107 -2.092105  0.730896
qux  13  0.707107 -0.773142  0.008552

In [37]:
# name our lambda columns
(grouped
     .aggregate([np.sum, lambda x: np.std(x, ddof=1)])
     .rename(columns={"sum": "sum", "<lambda>": "custom_std"}))

A                    B           
    sum custom_std       sum custom_std
X                                      
bar   6   1.290994  1.274463   0.644282
foo   9   0.707107 -2.092105   0.730896
qux  13   0.707107 -0.773142   0.008552

In [54]:
# use lambdas to filter the dataframe
df = pd.DataFrame({'A': np.arange(8), 'B': list('xxxyyddy')})
df

,A,B
0,0,x
1,1,x
2,2,x
3,3,y
4,4,y
5,5,d
6,6,d
7,7,y


In [57]:
# find rows where the value of B has more than 2 instances
df.groupby('B').filter(lambda x: len(x) > 2)

,A,B
0,0,x
1,1,x
2,2,x
3,3,y
4,4,y
7,7,y


In [58]:
grouped['A'].apply(lambda x: x.describe())

X         
bar  count    4.000000
     mean     1.500000
     std      1.290994
     min      0.000000
     25%      0.750000
     50%      1.500000
     75%      2.250000
     max      3.000000
foo  count    2.000000
     mean     4.500000
     std      0.707107
     min      4.000000
     25%      4.250000
     50%      4.500000
     75%      4.750000
     max      5.000000
qux  count    2.000000
     mean     6.500000
     std      0.707107
     min      6.000000
     25%      6.250000
     50%      6.500000
     75%      6.750000
     max      7.000000
Name: A, dtype: float64

In [60]:
# more generally, we apply() a function to an axis of a dataframe
# i.e. axis-wise apply

df = pd.DataFrame(
    {"A": [16] * 3, "B": [25] * 3}
)
df

,A,B
0,16,25
1,16,25
2,16,25


In [61]:
df.apply(np.sqrt)

,A,B
0,4.0,5.0
1,4.0,5.0
2,4.0,5.0


In [71]:
df.apply(lambda x: (x[0], x[1]), axis=1, result_type="expand")

,0,1
0,16,25
1,16,25
2,16,25


In [72]:
# element-wise apply
df.applymap(lambda x: x+5)

,A,B
0,21,30
1,21,30
2,21,30


In [75]:
# function chaining
(df.pipe(lambda x: x +5)
     .pipe(lambda x: x * 2))

,A,B
0,42,60
1,42,60
2,42,60


# Cleaning up misaggregated statistics and bad pivot tables

In [91]:
# pivot tables?
# dummy data setup
import pandas.util.testing as tm
df = pd.melt(tm.makeTimeDataFrame().reset_index(), id_vars=["index"])

In [87]:
# we can inspect variables one by one
df[df['variable'] == 'A']

,index,variable,value
0,2000-01-03,A,0.270418
1,2000-01-04,A,0.167347
2,2000-01-05,A,1.470468


In [88]:
df.pivot(index='index', columns='variable', values='value')

variable,A,B,C,D
index,,,,
2000-01-03,0.270418,0.142324,-0.994671,-0.732397
2000-01-04,0.167347,0.859365,-2.700172,-1.728973
2000-01-05,1.470468,1.554665,0.773388,-1.318421


In [94]:
df['value_2'] = df['value'] / 2
df

,index,variable,value,value_2
0,2000-01-03,A,1.878004,0.939002
1,2000-01-04,A,-0.486189,-0.243095
2,2000-01-05,A,1.661009,0.830504
3,2000-01-03,B,0.586774,0.293387
4,2000-01-04,B,-2.487804,-1.243902
5,2000-01-05,B,-0.113289,-0.056644
6,2000-01-03,C,2.085263,1.042632
7,2000-01-04,C,-0.141938,-0.070969
8,2000-01-05,C,-1.308571,-0.654285
9,2000-01-03,D,-0.214123,-0.107061


In [95]:
df.pivot(index='index', columns='variable')

value                                 value_2            \
variable           A         B         C         D         A         B   
index                                                                    
2000-01-03  1.878004  0.586774  2.085263 -0.214123  0.939002  0.293387   
2000-01-04 -0.486189 -2.487804 -0.141938 -0.898349 -0.243095 -1.243902   
2000-01-05  1.661009 -0.113289 -1.308571  2.632128  0.830504 -0.056644   

                                
variable           C         D  
index                           
2000-01-03  1.042632 -0.107061  
2000-01-04 -0.070969 -0.449174  
2000-01-05 -0.654285  1.316064

In [96]:
df = pd.DataFrame(
    {'A': np.arange(8), 'B': np.random.randn(8)},
    index=pd.MultiIndex.from_arrays(
          [['bar', 'bar', 'bar', 'bar', 'foo', 'foo', 'qux', 'qux'],
          ['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux']],
        names=['X', 'Y'])
)
df

A         B
X   Y               
bar bar  0 -0.232061
    bar  1 -0.002921
    baz  2  1.441982
    baz  3 -0.014129
foo foo  4 -0.363113
    foo  5  0.148242
qux qux  6 -1.373691
    qux  7  0.824880

In [98]:
df.stack()

X    Y     
bar  bar  A    0.000000
          B   -0.232061
          A    1.000000
          B   -0.002921
     baz  A    2.000000
          B    1.441982
          A    3.000000
          B   -0.014129
foo  foo  A    4.000000
          B   -0.363113
          A    5.000000
          B    0.148242
qux  qux  A    6.000000
          B   -1.373691
          A    7.000000
          B    0.824880
dtype: float64